# Running duqtools with docker containers

This Tutorial uses docker to create and submit runs which will be analyzed with duqtools. 
This notebook uses the `bash_kernel` which can be installed via pip: https://github.com/takluyver/bash_kernel


## Getting started

### Needed Packages

- The first thing we need is to install duqtools, if you haven't done this yet you can install it via `pip install duqtools`
- We will also need access to the source of duqtools, it can be downloaded from https://github.com/duqtools/duqtools, This notebook was created with version `1.5.0` of duqtools
- docker
    - We also need access to the following images (or equivalent): `jintrac-imas`. which can be found here https://gitlab.eufus.psnc.pl/containerization/jintrac/jintrac-imas-installer
- etc...

### Some example data

In this tutorial we will use example data, as IMAS is non-free you will have to get your IMAS data through your preferred supplier. We assume that the system is run with JINTRAC (since JINTRAC has a docker container available), and we use a simple JINTRAC template, which you are of course free to substitute for your own


In [1]:
!rm -rf example
!mkdir -p example/template
# We assume that the duqtools source is available and that we are in the docs/examples folder
!cp -r ../../tests/test_data/template_model/* example/template

### Some imas data, substitute with your own, as we are not allowed to provide it

In [2]:
!cp -r ../../../containerized_runs/imasdb example

In [3]:
# A silly script to support venvs
with open("example/entry.sh", "w") as f:
    f.write("""#!/bin/bash
set -e
. /etc/profile.d/modules.sh
module load IMAS
module load fc2k

if [ -d .venv ]; then
 . .venv/bin/activate
fi

exec "${@}"
""")
!chmod +x example/entry.sh

In [4]:
# A silly script to support venvs
with open("example/duqtools.yaml", "w") as f:
    f.write("""system: jetto
create:
  template: ./template #TODO edit to directory where initial run is stored
  template_data:
    user: "/example/imasdb"
    db: "jet"
    shot: 123
    run: 1
  dimensions:
  - operator: multiply
    scale_to_error: false
    values: [1.1, 1.2, 1.3]
    variable: t_e
  - operator: multiply
    scale_to_error: false
    values: [1.1, 1.2, 1.3]
    variable: zeff
  sampler:
    method: latin-hypercube
    n_samples: 3
submit:
  submit_system: docker
""")

In [5]:
!tree example

example
├── duqtools.yaml
├── entry.sh
├── imasdb
│   └── jet
│       └── 3
│           └── 0
│               ├── ids_1230001.characteristics
│               ├── ids_1230001.datafile
│               └── ids_1230001.tree
└── template
    ├── jetto.eqrestart
    ├── jetto.ex
    ├── jetto.in
    ├── jetto.jset
    ├── jetto.sgrid
    ├── jetto.sin
    ├── rjettov
    └── utils_jetto

5 directories, 13 files


In [6]:
# Get the image and tag it
!docker login gitlab.eufus.psnc.pl:5050
!docker pull gitlab.eufus.psnc.pl:5050/containerization/jintrac/jintrac-imas-installer/jintrac-imas:latest
!docker tag gitlab.eufus.psnc.pl:5050/containerization/jintrac/jintrac-imas-installer/jintrac-imas:latest jintrac-imas:latest

Authenticating with existing credentials...
WARNING! Your password will be stored unencrypted in /home/vikko/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
latest: Pulling from containerization/jintrac/jintrac-imas-installer/jintrac-imas
Digest: sha256:1b99ea457ba42bdd6809ef0069bf444ac2f8c3a1013ae9fe596da6f15987d584
Status: Image is up to date for gitlab.eufus.psnc.pl:5050/containerization/jintrac/jintrac-imas-installer/jintrac-imas:latest
gitlab.eufus.psnc.pl:5050/containerization/jintrac/jintrac-imas-installer/jintrac-imas:latest


In [7]:
import docker
import os
niceprint = lambda s: print(str(s.output,"utf-8"))
run_inside = lambda container, cmd: container.exec_run(" ".join(['/example/entry.sh',cmd]))

In [8]:
client = docker.from_env()
container = client.containers.run('jintrac-imas', 
                                  working_dir="/example", 
                                  user=os.getuid(),
                                  detach=True, 
                                  entrypoint="/example/entry.sh",
                                  command="tail -f /dev/null",
                                  volumes=[os.getcwd()+"/example:/example"],
                                  auto_remove=True)

In [9]:
run_inside(container,'python3 -m venv .venv');

In [10]:
niceprint(run_inside(container,'python3 -m pip install duqtools -q'))

You should consider upgrading via the '/example/.venv/bin/python3 -m pip install --upgrade pip' command.



In [11]:
niceprint(run_inside(container,'duqtools create --force --yes')) 

15:17:16 [WARNING] Python module 'omas' not found. Submodule 'jams' needs it @jams.py:14
15:17:16 [WARNING] Python module 'netCDF4' not found. Submodule 'transp' needs it @transp.py:25
15:17:16 [WARNING] Python module 'tkinter' not found. Submodule 'tkinter_helpers' needs it @tkinter_helpers.py:16

Operations in the Queue:
- Creating run : /example/run_0000
- Creating run : /example/run_0001
- Creating run : /example/run_0002
Applying Operations
21

  0%|          | 0/21 [00:00<?, ?it/s]
Creating run : /example/run_0000: it/s]
Progress:  10%|▉         | 2/21 [00:00<00:04,  4.20it/s]
Creating run : /example/run_0001: 2,  7.73it/s]
Progress:  38%|███▊      | 8/21 [00:00<00:01,  8.94it/s]
Creating run : /example/run_0002: 01,  9.87it/s]
Progress:  67%|██████▋   | 14/21 [00:01<00:00,  9.59it/s]
Creating run : /example/run_0002: 00, 12.15it/s]
Progress: 100%|██████████| 21/21 [00:01<00:00, 11.97it/s]



In [12]:
#!cd example && duqtools submit --yes